# 配置信息

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

model_name_or_path = '/home/abc/mydisk/model/chatglm3-6b'  # 模型路径
peft_model_path = "/home/abc/mydisk/model/chatglm3-6b/out/checkpoint-5000/"  # 微调模型路径
training_tag="ChatGLM3-6B(checkpoint-5000, train_data: HasturOfficial/adgen, 20k/115k)" # 微调情况

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

config = PeftConfig.from_pretrained(peft_model_path)

In [3]:
# 加载模型，tokenizer
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True)

qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

base_model.requires_grad_(False)
base_model.eval()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): lora.Linear4bit(
              (base_layer): Linear4bit(in_features=4096, out_features=4608, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.05, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=4096, out_features=8, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=8, out_features=4608, bias=False)
              )
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
            )
            (

In [4]:
! nvidia-smi

Wed Jul 10 18:25:05 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10                     Off |   00000000:00:08.0 Off |                    0 |
|  0%   43C    P0             61W /  150W |    4302MiB /  23028MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
# 比较方法，比较微调前后的结果
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

# 开始比较

In [6]:
base_response, ft_response = compare_chatglm_results("为这款裙子生成广告语。'类型#裙*版型#显瘦*材质#网纱*风格#性感*裙型#百褶*裙下摆#压褶*裙长#连衣裙*裙衣门襟#拉链*裙衣门襟#套头*裙款式#拼接*裙款式#拉链*裙款式#木耳边*裙款式#抽褶*裙款式#不规则'", base_model, qlora_model, training_tag)

问题：为这款裙子生成广告语。'类型#裙*版型#显瘦*材质#网纱*风格#性感*裙型#百褶*裙下摆#压褶*裙长#连衣裙*裙衣门襟#拉链*裙衣门襟#套头*裙款式#拼接*裙款式#拉链*裙款式#木耳边*裙款式#抽褶*裙款式#不规则'

原始输出：
这款网纱拼接套头连衣裙，时尚的木耳边装饰，尽显甜美可爱气息。不规则的压褶设计，打造时髦感。不规则的裙摆，展现纤细双腿。加上不规则的百褶裙摆，修身显瘦，更显女性气质。网纱百褶裙摆，时尚浪漫，性感迷人。精致的拉链设计，穿脱方便。


微调后（ChatGLM3-6B(checkpoint-5000, train_data: HasturOfficial/adgen, 20k/115k)）：
[gMASK] sop 为这款裙子生成广告语。'类型#裙*版型#显瘦*材质#网纱*风格#性感*裙型#百褶*裙下摆#压褶*裙长#连衣裙*裙衣门襟#拉链*裙衣门襟#套头*裙款式#拼接*裙款式#拉链*裙款式#木耳边*裙款式#抽褶*裙款式#不规则' 这款连衣裙采用网纱拼接，透视感十足，性感迷人。不规则的木耳边装饰，甜美可爱。拼接的压褶设计，层次感十足，修身显瘦。套头式设计，方便穿脱。百褶裙摆，飘逸灵动。拉链设计，方便穿脱。


In [7]:
base_response, ft_response = compare_chatglm_results("我想买一条夏天穿的裙子，有什么好建议么", base_model, qlora_model, training_tag)

问题：我想买一条夏天穿的裙子，有什么好建议么

原始输出：
夏天穿衣要注重轻盈与透气，所以选择一款轻薄且透气的裙子是不错的选择。

1. 颜色选择:

裙子颜色选择上可以尝试选择明快的颜色，如粉色、蓝色、绿色等，明亮的颜色可以更好的展现活力感，轻松减龄。

2. 材质选择:

选择轻薄且透气的面料，如棉布、雪纺等，可以更好的营造夏日的清爽感。

3. 款式选择:

裙子款式选择上可以尝试选择轻便的款式，如吊带裙、裙裤等，更加轻松。

4. 面料:

雪纺、薄款棉布、化纤棉布等轻薄透气面料适合夏天穿着，垂感好，透气性好，柔软亲肤。


微调后（ChatGLM3-6B(checkpoint-5000, train_data: HasturOfficial/adgen, 20k/115k)）：
[gMASK] sop 我想买一条夏天穿的裙子，有什么好建议么? 
 夏天穿裙子是再正常不过的事情了，不过要选一条既好看又舒适的裙子，就有点难度了。下面是一些值得推荐的夏季裙子，希望对你有所帮助。

1. 轻薄透气：夏季气温高，所以要选择布料柔软、透气性好的裙子，这样穿起来才舒适。像雪纺、薄棉布、化纤布等都是不错的选择。

2. 颜色选择：夏季气温高，颜色选择要浅，这样看起来才清爽。浅色系的裙子会显得人更有活力，而且不显胖。深色系的裙子则显得沉稳，适合成熟的女性。

3. 款式选择：夏季裙子要选择舒适、透气的款式，不要穿太紧身的裙子。吊带裙、裙摆宽松的裙子都是不错的选择，而且看起来很清爽。

4. 材质选择：夏季气温高，要选择透气性好的面料，雪纺、薄棉布、化纤布都是不错的选择。


In [8]:
base_response, ft_response = compare_chatglm_results("'类型#上衣*版型#显瘦*图案#刺绣*衣样式#衬衫*衣袖型#泡泡袖*衣款式#抽绳*风格#简约'", base_model, qlora_model, training_tag)

问题：'类型#上衣*版型#显瘦*图案#刺绣*衣样式#衬衫*衣袖型#泡泡袖*衣款式#抽绳*风格#简约'

原始输出：
这款衬衫的设计非常独特，采用别致的泡泡袖设计，上身效果非常显瘦，彰显出甜美可爱的气质。加上简约大方的直筒版型，修饰身形。加上刺绣点缀，打破简约的风格，更显精致，穿着非常时尚。加上抽绳的设计，更是时尚个性。


微调后（ChatGLM3-6B(checkpoint-5000, train_data: HasturOfficial/adgen, 20k/115k)）：
[gMASK] sop '类型#上衣*版型#显瘦*图案#刺绣*衣样式#衬衫*衣袖型#泡泡袖*衣款式#抽绳*风格#简约' 简约的衬衫版型，上身效果干净利落，穿着舒适。袖子采用抽绳设计，可随意调节，穿着舒适。袖子采用刺绣设计，精致美观，彰显品质。泡泡袖设计，遮肉显瘦，穿着时尚。
